Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

Используется устройство: cuda


In [3]:
websites_df = pd.read_csv('websites_updated.csv')
questions_df = pd.read_csv('questions_clean.csv')
websites_df.set_index('web_id', inplace=True)
questions_df.set_index('q_id', inplace=True)
websites_df = websites_df.drop(1938)

In [4]:
websites_df.tail()

,url,kind,title,text
web_id,,,,
1933,https://alfabank.ru/get-money/land/credit-holi...,html,Кредитные каникулы — Альфа-Банк,Кредитные каникулы — это возможность временно ...
1934,https://alfabank.ru/help/t/retail/alfaforbusin...,html,Как вернуть деньги покупателю и как рассчитыва...,Возврат денег покупателю можно оформить через ...
1935,https://alfabank.ru/help/articles/investments/...,html,Как вывести деньги с брокерского счёта — Альфа...,Вывести деньги с брокерского счёта можно на ка...
1936,https://alfabank.ru/make-money/investments/hel...,html,Пополнение и вывод средств — Альфа-Инвестиции,Вывести деньги с брокерского счёта можно на сл...
1937,https://alfabank.ru/everyday/smart/,html,Альфа-Смарт — подписка Альфа-Банка,"Альфа-Смарт — семейная подписка, запущенная в ..."


Предобработка и токенизация

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3

In [6]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\AdminWin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AdminWin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stop_words = set(stopwords.words('russian'))
morph = pymorphy3.MorphAnalyzer()

In [8]:
processed_texts = []
for idx, row in tqdm(websites_df.iterrows(), total=len(websites_df)):
    full_text = f"{row['title']} {row['text']}"
    text = full_text.lower()
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = word_tokenize(text, language='russian')
    
    processed_tokens = []
    for token in tokens:
        if (token not in stop_words and 
            len(token) > 2 and 
            token.isalpha()):
            lemma = morph.parse(token)[0].normal_form
            processed_tokens.append(lemma)
    
    processed_text = ' '.join(processed_tokens)
    processed_texts.append(processed_text)

websites_df['processed_text'] = processed_texts

100%|██████████| 1937/1937 [01:24<00:00, 22.82it/s] 


In [9]:
websites_df.tail()

,url,kind,title,text,processed_text
web_id,,,,,
1933,https://alfabank.ru/get-money/land/credit-holi...,html,Кредитные каникулы — Альфа-Банк,Кредитные каникулы — это возможность временно ...,кредитный каникулы альфа банк кредитный канику...
1934,https://alfabank.ru/help/t/retail/alfaforbusin...,html,Как вернуть деньги покупателю и как рассчитыва...,Возврат денег покупателю можно оформить через ...,вернуть деньга покупатель рассчитываться комис...
1935,https://alfabank.ru/help/articles/investments/...,html,Как вывести деньги с брокерского счёта — Альфа...,Вывести деньги с брокерского счёта можно на ка...,вывести деньга брокерский счёт альфа банк выве...
1936,https://alfabank.ru/make-money/investments/hel...,html,Пополнение и вывод средств — Альфа-Инвестиции,Вывести деньги с брокерского счёта можно на сл...,пополнение вывод средство альфа инвестиция выв...
1937,https://alfabank.ru/everyday/smart/,html,Альфа-Смарт — подписка Альфа-Банка,"Альфа-Смарт — семейная подписка, запущенная в ...",альфа смарт подписка альфа банк альфа смарт се...


Чанкирование

In [10]:
from langchain_text_splitters import TokenTextSplitter
import hashlib
import tiktoken

In [11]:
token_splitter = TokenTextSplitter(
    chunk_size=256,           # Количество токенов в чанке
    chunk_overlap=64,         # Перекрытие между чанками
    encoding_name="cl100k_base"  # Кодировка для подсчета токенов (используется в GPT)
)

In [12]:
all_chunks = []
chunk_metadata = []

for idx, row in tqdm(websites_df.iterrows(), total=len(websites_df)):
    if len(row['processed_text'].strip()) < 50:
        continue
    
    chunks = token_splitter.split_text(row['processed_text'])
    
    for chunk_idx, chunk in enumerate(chunks):
        if len(chunk.strip()) > 30:
            chunk_id = hashlib.md5(f"{idx}_{chunk_idx}".encode()).hexdigest()[:8]
            
            all_chunks.append(chunk)
            chunk_metadata.append({
                'chunk_id': chunk_id,
                'web_id': idx,
                'chunk_index': chunk_idx,
                'original_url': row.get('url', ''),
                'text_length': len(chunk),
            })

print(f"Создано {len(all_chunks)} чанков из {len(websites_df)} документов")
print(f"Средняя длина чанка: {np.mean([m['text_length'] for m in chunk_metadata]):.1f} символов")

100%|██████████| 1937/1937 [00:01<00:00, 1005.61it/s]

Создано 22006 чанков из 1937 документов
Средняя длина чанка: 597.5 символов


Векторизация

In [13]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [14]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True}
)

C:\Users\AdminWin\AppData\Local\Temp\ipykernel_16832\3961618539.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [15]:
documents = []
for i, chunk_text in enumerate(all_chunks):
    doc = Document(
        page_content=chunk_text,
        metadata=chunk_metadata[i]
    )
    documents.append(doc)

In [16]:
vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

Векторный поиск

In [17]:
from rank_bm25 import BM25Okapi

In [18]:
texts_for_bm25 = [doc.page_content for doc in documents]
tokenized_corpus = [text.split() for text in texts_for_bm25]
bm25_index = BM25Okapi(tokenized_corpus)

In [29]:
def hybrid_search(query, vector_store, bm25_index, documents, top_k=5, alpha=0.7):
    """Гибридный поиск: комбинация семантического (FAISS) и ключевого (BM25)"""
    processed_query = query.lower()
    processed_query = re.sub(r'<[^>]+>', '', processed_query)
    processed_query = re.sub(r'[^\w\s]', ' ', processed_query)
    processed_query = re.sub(r'\s+', ' ', processed_query).strip()
    
    tokens = word_tokenize(processed_query, language='russian')
    processed_tokens = []
    for token in tokens:
        if (token not in stop_words and len(token) > 2 and token.isalpha()):
            lemma = morph.parse(token)[0].normal_form
            processed_tokens.append(lemma)
    
    processed_query = ' '.join(processed_tokens)
    
    semantic_results = vector_store.similarity_search_with_score(
        processed_query, 
        k=top_k * 3
    )
    
    semantic_scores = {}
    for doc, score in semantic_results:
        chunk_id = doc.metadata['chunk_id']
        similarity = 1 - score
        if chunk_id not in semantic_scores:
            semantic_scores[chunk_id] = similarity
    
    # Ключевой поиск через BM25
    tokenized_query = processed_query.split()
    if tokenized_query:
        bm25_scores = bm25_index.get_scores(tokenized_query)
        
        keyword_scores = {}
        for idx, score in enumerate(bm25_scores):
            chunk_id = documents[idx].metadata['chunk_id']
            if chunk_id not in keyword_scores:
                keyword_scores[chunk_id] = score
    else:
        keyword_scores = {}
    
    # Нормализация scores
    if semantic_scores:
        max_semantic = max(semantic_scores.values())
        for chunk_id in semantic_scores:
            semantic_scores[chunk_id] /= max_semantic if max_semantic > 0 else 1

    if keyword_scores:
        max_keyword = max(keyword_scores.values())
        for chunk_id in keyword_scores:
            keyword_scores[chunk_id] /= max_keyword if max_keyword > 0 else 1

    # Комбинирование scores
    combined_scores = {}
    all_chunk_ids = set(list(semantic_scores.keys()) + list(keyword_scores.keys()))
    
    for chunk_id in all_chunk_ids:
        semantic_score = semantic_scores.get(chunk_id, 0)
        keyword_score = keyword_scores.get(chunk_id, 0)
        combined_score = alpha * semantic_score + (1 - alpha) * keyword_score
        combined_scores[chunk_id] = combined_score
    
    # Возвращаем топ-K документов
    top_chunk_ids = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:top_k*3]
    return [chunk_id for chunk_id, score in top_chunk_ids]

Re-Ranking

In [20]:
from sentence_transformers import CrossEncoder

In [21]:
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [22]:
def search_with_reranking(query, vector_store, bm25_index, documents, top_k=5):
    """Поиск с реранжированием"""
    candidate_chunk_ids = hybrid_search(query, vector_store, bm25_index, documents, top_k=top_k * 3)
    
    candidate_texts = []
    chunk_id_to_web_id = {}  # Маппинг chunk_id → web_id
    
    for chunk_id in candidate_chunk_ids:
        doc = next((d for d in documents if d.metadata.get('chunk_id') == chunk_id), None)
        candidate_texts.append(doc.page_content)
        chunk_id_to_web_id[chunk_id] = doc.metadata['web_id']
    
    # Оценка релевантности пар (запрос, документ)
    pairs = [[query, doc_text] for doc_text in candidate_texts]
    scores = reranker.predict(pairs)
    
    # Сортировка по убыванию релевантности
    scored_candidates = []
    for i, (chunk_id, score) in enumerate(zip(candidate_chunk_ids, scores)):
        web_id = chunk_id_to_web_id.get(chunk_id)
        scored_candidates.append((web_id, score, chunk_id))
    
    # Сортировка по убыванию релевантности
    scored_candidates.sort(key=lambda x: x[1], reverse=True)
    
    # Убираем дубликаты web_id, оставляя только лучший результат для каждого web_id
    unique_web_ids = []
    seen_web_ids = set()
    
    for web_id, score, chunk_id in scored_candidates:
        if web_id not in seen_web_ids:
            unique_web_ids.append(web_id)
            seen_web_ids.add(web_id)
        if len(unique_web_ids) >= top_k:
            break
    
    return unique_web_ids[:top_k]

Выполнение поиска

In [30]:
results = []

for q_id, row in tqdm(questions_df.iterrows(), total=len(questions_df)):
    query = row['query']
    
    top_web_ids = search_with_reranking(
        query, vector_store, bm25_index, documents, top_k=5
    )
    
    web_list_str = f"[{', '.join(map(str, top_web_ids))}]"
    results.append({'q_id': q_id, 'web_list': web_list_str})

# Сохранение результатов
submission_df = pd.DataFrame(results)
with open('submit.csv', 'w', encoding='utf-8') as f:
    f.write('q_id,web_list\n')
    for result in results:
        f.write(f'{result["q_id"]},"{result["web_list"]}"\n')

submission_df.head()

100%|██████████| 6977/6977 [48:02<00:00,  2.42it/s] 


,q_id,web_list
0,1,"[1557, 900, 835, 372, 1704]"
1,2,"[1557, 1239, 108, 415, 1798]"
2,3,"[1704, 1758, 1760, 344, 1705]"
3,4,"[1038, 1032, 343, 1042, 1760]"
4,5,"[199, 175, 1039, 1583, 164]"
